## Subword exercise of chula NLP2025

In this exercise, we will learn how to train our own subword tokenizers with different algorithms: BPE and Unigram. We will use sentencepiece, a library from Google to help create our tokenizers.

## Upload data

In [3]:
!curl -LO https://github.com/Knight-H/thai-lm/raw/refs/heads/master/data/pra-apai-manee-ch1-50.txt
!curl -LO https://github.com/Knight-H/thai-lm/raw/refs/heads/master/data/kratoo-40000000-40002000.jsonl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 3155k  100 3155k    0     0  1950k      0  0:00:01  0:00:01 --:--:-- 7733k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  2 2898k    2 64620    0     0  38054      0  0

In [4]:
import sentencepiece as spm
import io
import json

## kratoo data

In [12]:
pantip_text = []
with open('kratoo-40000000-40002000.jsonl','r') as json_file:
    json_list = list(json_file)
    for i in json_list:
        result = json.loads(i)
        pantip_text.append(f"{result['title']}\n{result['content']}\n")
pantip_text[0]

'ใครรู้จักคนนี้บ้าง\nคือเราคุ้นๆคนนี้บ้างเลยอยากรู้ว่าคนนี้ชื่อว่าอะไร\n'

## pra_apai_manee_data

In [ ]:
with open("pra-apai-manee-ch1-50.txt") as f:
  pra_apai_manee_data = f.readlines()
pra_apai_manee_data[2]

'อันกรุงไกรใหญ่ยาวสิบเก้าโยชน์\tภูเขาโขดเป็นกำแพงบุรีศรี\n'

## Data splitting

In [90]:
pantip_train_text = pantip_text[:int(len(pantip_text)*0.8)]
pantip_test_text = pantip_text[int(len(pantip_text)*0.8):]

pam_train_text = pra_apai_manee_data[:int(len(pra_apai_manee_data)*0.8)] #pam = pra_apai_manee
pam_test_text = pra_apai_manee_data[int(len(pra_apai_manee_data)*0.8):]

In [21]:
pantip_train_text[0]

'ใครรู้จักคนนี้บ้าง\nคือเราคุ้นๆคนนี้บ้างเลยอยากรู้ว่าคนนี้ชื่อว่าอะไร\n'

In [91]:
with open("pantip_train_text.txt", "w", encoding="utf-8") as f:
    for line in pantip_train_text:
        f.write(line.strip() + "\n")

In [92]:
with open("pam_train_text.txt", "w", encoding="utf-8") as f:
    for line in pam_train_text:
        f.write(line.strip() + "\n")

## Create your own dictionary

In [93]:
import sentencepiece as spm

## Unigram tokenizer

In [94]:
spm.SentencePieceTrainer.train(
    input='pam_train_text.txt', 
    model_prefix='pam_unigram',#output file model,vocab
    vocab_size=2000,
    model_type='unigram',
    pad_id=0, pad_piece='[PAD]',
    unk_id=1, unk_piece='[UNK]',
    bos_id=2, bos_piece='[BOS]',
    eos_id=3, eos_piece='[EOS]',
    user_defined_symbols='[SEP],[CLS],[MASK]'
)

In [95]:
sp_ui = spm.SentencePieceProcessor()
sp_ui.load('pam_unigram.model')

True

In [96]:
len(sp_ui.encode('อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม', out_type=str))


23

In [98]:
sp_ui.encode('อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม', out_type=str)

['▁',
 'อรุณ',
 'สวัสดิ์',
 '▁',
 'ฉัน',
 'เอา',
 'มเหสี',
 'มาหา',
 'ม',
 '▁',
 'ส',
 'ว',
 'ั',
 'ส',
 'ดี',
 '▁',
 'ประเทศ',
 'ไทย',
 'สบาย',
 'ดี',
 'ไ',
 'ห',
 'ม']

## BPE

In [100]:
spm.SentencePieceTrainer.train(
    input="pam_train_text.txt",
    model_prefix="pantip_bpe",
    vocab_size=2000,      # adjust as needed
    model_type="bpe",     # <-- here is the difference!
    pad_id=0, pad_piece='[PAD]',
    unk_id=1, unk_piece='[UNK]',
    bos_id=2, bos_piece='[BOS]',
    eos_id=3, eos_piece='[EOS]',
    user_defined_symbols='[SEP],[CLS],[MASK]'
)

In [101]:
sp_bpe = spm.SentencePieceProcessor()
sp_bpe.load("pantip_bpe.model")

True

In [102]:
len(sp_bpe.encode('อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม', out_type=str))


21

In [103]:
sp_bpe.encode('อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม', out_type=str)


['▁อ',
 'รุณ',
 'สวัสดิ์',
 '▁ฉัน',
 'เอ',
 'าม',
 'เหสี',
 'มา',
 'หาม',
 '▁ส',
 'ว',
 'ัส',
 'ดี',
 '▁ประ',
 'เทศ',
 'ไท',
 'ย',
 'สบาย',
 'ดี',
 'ไ',
 'หม']

In [104]:
unigram_vocabs = [sp_ui.id_to_piece(id) for id in range(sp_ui.get_piece_size())]
" | ".join(unigram_vocabs[:500])
unigram_vocabs

['[PAD]',
 '[UNK]',
 '[BOS]',
 '[EOS]',
 '[SEP]',
 '[CLS]',
 '[MASK]',
 '▁',
 'า',
 'มา',
 'ให้',
 'จะ',
 'ม',
 'เ',
 'ไป',
 'ก',
 'ไม่',
 'ว่า',
 '๏',
 'ฯ',
 'น',
 'บ',
 'ด',
 'ร',
 'ย',
 'ง',
 'ส',
 '▁จะ',
 'เป็น',
 'ใจ',
 'ค',
 'ห',
 'ที่',
 'อยู่',
 'อ',
 'ได้',
 'พระ',
 'ท',
 'ก็',
 'ว',
 '▁แล้ว',
 'ตาม',
 '▁พระ',
 'ุ',
 'ล',
 'กัน',
 'ช',
 'แ',
 'พ',
 'นาง',
 'ป',
 'จ',
 'ใน',
 'มี',
 'ด้วย',
 'แล้ว',
 'เห็น',
 'เข้า',
 'เจ้า',
 '▁ให้',
 'แต่',
 'ี',
 'เหมือน',
 'คิด',
 'โ',
 '▁จึง',
 'ต',
 '▁นาง',
 'ิ',
 'ะ',
 'ถึง',
 'องค์',
 'รัก',
 'ข',
 'กับ',
 'ทรง',
 'หน้า',
 'ดู',
 'การ',
 'ความ',
 'หา',
 'ลง',
 'ทํา',
 'น้อง',
 'พา',
 'คน',
 '▁แต่',
 'ทั้ง',
 'นั่ง',
 'ลา',
 '▁เห็น',
 'แม่',
 'ไว้',
 'รับ',
 'ลูก',
 'ดัง',
 'เสีย',
 'ฟัง',
 '▁พอ',
 'กระ',
 'เขา',
 'ประ',
 'ถ',
 'นี้',
 'พล',
 '▁ฝ่าย',
 '▁ทั้ง',
 'ดี',
 'น้ํา',
 'รู้',
 'ทัพ',
 '▁ไม่',
 'รา',
 'อย่า',
 'นั้น',
 'ขึ้น',
 '์',
 'นึก',
 'พี่',
 'หรือ',
 'ซ',
 'จน',
 'ข้าง',
 'ตาย',
 'รบ',
 '▁ถึง',
 'กลับ',
 'อา',
 'ศ',
 'ตัว

These are some of your vocabs. Note that you will see "▁" (U+2581) in every type of tokenizer in SentencePiece since it makes it possible to perform detokenization (unsplit your sentences) without relying on language-specific resources.

Another important concept to know of is User-defined symbols. These special symbols are reserved for a special purpose (e.g., the <MASK> token used in BERT) and will always be tokenized into one token.

Refer to the documentation for ways to add these special tokens to your tokenizer.

## Compare between datasets

using BPE

In [105]:
spm.SentencePieceTrainer.train(
    input='pantip_train_text.txt', 
    model_prefix='pantip_unigram',#output file model,vocab
    vocab_size=2000,
    model_type='bpe',
    pad_id=0, pad_piece='[PAD]',
    unk_id=1, unk_piece='[UNK]',
    bos_id=2, bos_piece='[BOS]',
    eos_id=3, eos_piece='[EOS]',
    user_defined_symbols='[SEP],[CLS],[MASK]'
)

In [106]:
sp_bpe = spm.SentencePieceProcessor()
sp_bpe.load("pantip_bpe.model")
sp_pantip = spm.SentencePieceProcessor()
sp_pantip.load("pantip_unigram.model")

True

In [109]:
print(len(sp_pantip.encode('อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม', out_type=str)))
print(len(sp_bpe.encode('อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม', out_type=str)))

25
21


In [ ]:
sp_pantip.encode('อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม', out_type=str) 

['▁อ',
 'รุ',
 'ณ',
 'ส',
 'วัส',
 'ดิ',
 '์',
 '▁',
 'ฉัน',
 'เอ',
 'าม',
 'เห',
 'สี',
 'มา',
 'ห',
 'าม',
 '▁สวัส',
 'ดี',
 '▁',
 'ประเทศ',
 'ไทย',
 'สบ',
 'าย',
 'ดี',
 'ไหม']

In [107]:
sp_bpe.encode('อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม', out_type=str)

['▁อ',
 'รุณ',
 'สวัสดิ์',
 '▁ฉัน',
 'เอ',
 'าม',
 'เหสี',
 'มา',
 'หาม',
 '▁ส',
 'ว',
 'ัส',
 'ดี',
 '▁ประ',
 'เทศ',
 'ไท',
 'ย',
 'สบาย',
 'ดี',
 'ไ',
 'หม']